In [ ]:
# Install neqsim if needed (uncomment for Colab)
# !pip install neqsim

In [ ]:
from neqsim.thermo import fluid, TPflash, phaseenvelope
import matplotlib.pyplot as plt
import pandas as pd

## 1. Create a Natural Gas Mixture

We'll create a rich natural gas that exhibits both bubble point and dew point behavior.

In [ ]:
# Create a rich natural gas mixture
gas = fluid("srk")
gas.addComponent("nitrogen", 1.0, "mol%")
gas.addComponent("CO2", 2.0, "mol%")
gas.addComponent("methane", 75.0, "mol%")
gas.addComponent("ethane", 10.0, "mol%")
gas.addComponent("propane", 6.0, "mol%")
gas.addComponent("i-butane", 1.5, "mol%")
gas.addComponent("n-butane", 2.0, "mol%")
gas.addComponent("i-pentane", 1.0, "mol%")
gas.addComponent("n-pentane", 1.0, "mol%")
gas.addComponent("n-hexane", 0.5, "mol%")
gas.setMixingRule("classic")
gas.setMultiPhaseCheck(True)

print("Natural gas mixture created")
print(f"Total moles: {gas.getTotalNumberOfMoles():.4f}")

## 2. Calculate Phase Envelope

In [ ]:
# Set initial conditions and perform flash
gas.setTemperature(25.0, "C")
gas.setPressure(50.0, "bara")
TPflash(gas)

# Calculate phase envelope
print("Calculating phase envelope...")
envelope = phaseenvelope(gas)
print("Done!")

In [ ]:
# Extract data from envelope
temps = envelope.get("Tsat")  # Temperature in K
pressures = envelope.get("Psat")  # Pressure in bara

# Convert to Celsius
temps_c = [t - 273.15 for t in temps]

print(f"Number of envelope points: {len(temps)}")

## 3. Extract Cricondenbar and Cricondentherm

In [ ]:
# Get critical points
try:
    cricondenbar_T = envelope.get("cricondenbarT")[0] - 273.15
    cricondenbar_P = envelope.get("cricondenbarP")[0]
    cricondentherm_T = envelope.get("cricondenthermT")[0] - 273.15
    cricondentherm_P = envelope.get("cricondenthermP")[0]
    
    print(f"Cricondenbar (maximum pressure):")
    print(f"  P = {cricondenbar_P:.2f} bara at T = {cricondenbar_T:.1f}°C")
    print(f"\nCricondentherm (maximum temperature):")
    print(f"  T = {cricondentherm_T:.1f}°C at P = {cricondentherm_P:.2f} bara")
except Exception as e:
    print(f"Could not extract critical points: {e}")
    cricondenbar_T, cricondenbar_P = None, None
    cricondentherm_T, cricondentherm_P = None, None

## 4. Plot Phase Envelope

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(temps_c, pressures, 'b-', linewidth=2, label='Phase Envelope')

# Mark cricondenbar and cricondentherm
if cricondenbar_T is not None:
    plt.plot(cricondenbar_T, cricondenbar_P, 'ro', markersize=10, label=f'Cricondenbar ({cricondenbar_P:.1f} bara)')
if cricondentherm_T is not None:
    plt.plot(cricondentherm_T, cricondentherm_P, 'g^', markersize=10, label=f'Cricondentherm ({cricondentherm_T:.1f}°C)')

plt.xlabel('Temperature [°C]', fontsize=12)
plt.ylabel('Pressure [bara]', fontsize=12)
plt.title('Phase Envelope - Rich Natural Gas', fontsize=14)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.xlim(min(temps_c) - 10, max(temps_c) + 10)
plt.ylim(0, max(pressures) * 1.1)

# Add regions annotation
plt.annotate('Two-Phase\nRegion', xy=(sum(temps_c)/len(temps_c), max(pressures)/2),
             fontsize=12, ha='center')

plt.tight_layout()
plt.show()

## 5. Create DataFrame with Envelope Data

In [ ]:
# Create a DataFrame for easy analysis
df = pd.DataFrame({
    'Temperature [°C]': temps_c,
    'Pressure [bara]': pressures
})

# Show first and last points
print("Phase Envelope Data (first 10 points):")
df.head(10)

## 6. Compare Different Gas Compositions

In [ ]:
# Lean gas (less heavy ends)
lean_gas = fluid("srk")
lean_gas.addComponent("nitrogen", 2.0, "mol%")
lean_gas.addComponent("methane", 92.0, "mol%")
lean_gas.addComponent("ethane", 4.0, "mol%")
lean_gas.addComponent("propane", 2.0, "mol%")
lean_gas.setMixingRule("classic")
lean_gas.setMultiPhaseCheck(True)
lean_gas.setTemperature(25.0, "C")
lean_gas.setPressure(50.0, "bara")
TPflash(lean_gas)

lean_envelope = phaseenvelope(lean_gas)
lean_temps = [t - 273.15 for t in lean_envelope.get("Tsat")]
lean_pressures = lean_envelope.get("Psat")

print("Lean gas envelope calculated")

In [ ]:
# Plot comparison
plt.figure(figsize=(10, 7))
plt.plot(temps_c, pressures, 'b-', linewidth=2, label='Rich Gas')
plt.plot(lean_temps, lean_pressures, 'r--', linewidth=2, label='Lean Gas')

plt.xlabel('Temperature [°C]', fontsize=12)
plt.ylabel('Pressure [bara]', fontsize=12)
plt.title('Phase Envelope Comparison: Rich vs Lean Gas', fontsize=14)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nNote: Rich gas has higher cricondentherm due to heavier components")

## Summary

This notebook demonstrated:
- How to create fluid compositions in NeqSim
- How to calculate phase envelopes
- How to extract and plot cricondenbar/cricondentherm
- How composition affects the phase envelope shape

**Key concepts:**
- **Cricondenbar**: Maximum pressure on the phase envelope
- **Cricondentherm**: Maximum temperature on the phase envelope
- **Two-phase region**: Inside the envelope, both liquid and vapor coexist